## Model Train 1 - Tesis Javier-Uriel

### Importamos algunas librerías que nos serán útiles más adelante

In [1]:
import os
import json
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv

import tensorflow as tf
from tensorflow.keras import models, layers
assert (tf.__version__=='2.4.1'), 'Versión incorrecta de Tensorflow, por favor instale 2.4.1'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from IPython.display import clear_output

pd.set_option('display.max_columns', None) #Para mostrar todas las columnas

import gc #garbage collector
import gc; gc.enable()

Num GPUs Available:  1


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


### Leemos el Dataset

In [3]:
#Dataset solo movimientos en Z
rpm_list = ['RPM0', 'RPM1', 'RPM2', 'RPM3']
states_list_org = ["vz","az", "uvz",
                    "p", "q",
                    "wp", "wq", 
                    "ap", "aq"]
dataset_name = "Dataset_Z10_Alle"
directory = "../logs/Datasets/"+dataset_name
ORDER = 3
dfs = []
states_list=states_list_org.copy()

In [4]:
for filename in os.listdir(directory):
    if not filename.endswith(".csv"):
        continue
    df = pd.read_csv(os.path.join(directory, filename))
    a = []
    ## Desplazamos estados anteriores        
    for n in range(1,ORDER+1):
        for column in states_list:
            df[column+str(n)] = df[column].shift(periods=n, fill_value=0)
            a.append(column+str(n))
    dfs.append(df)
states_list+=a        
        
dataset = pd.concat(dfs)
dataset.head()

,timestamps,x,y,z,Q1,Q2,Q3,Q4,p,q,r,vx,vy,vz,wp,wq,wr,ax,ay,az,ap,aq,ar,RPM0,RPM1,RPM2,RPM3,ux,uy,uz,uvx,uvy,uvz,up,uq,ur,uwp,uwq,uwr,vz1,az1,uvz1,p1,q1,wp1,wq1,ap1,aq1,vz2,az2,uvz2,p2,q2,wp2,wq2,ap2,aq2,vz3,az3,uvz3,p3,q3,wp3,wq3,ap3,aq3
0,0.000000,0.0,0.0,49.999830,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.040833,0.0,0.0,0.0,0.0,0.0,-9.800000,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.004167,0.0,0.0,49.999562,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.064276,0.0,0.0,0.0,0.0,0.0,-5.626198,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,-0.040833,-9.800000,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.008333,0.0,0.0,49.999197,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.087714,0.0,0.0,0.0,0.0,0.0,-5.625162,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,-0.064276,-5.626198,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.040833,-9.800000,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.012500,0.0,0.0,49.998733,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.111148,0.0,0.0,0.0,0.0,0.0,-5.624082,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,-0.087714,-5.625162,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.064276,-5.626198,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.040833,-9.800000,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.016667,0.0,0.0,49.998173,0.0,0.0,0.0,1.0,0.0,-0.0,0.0,0.0,0.0,-0.134577,0.0,0.0,0.0,0.0,0.0,-5.622958,0.0,0.0,0.0,9440.3,9440.3,9440.3,9440.3,0.0,0.0,50.0,0.0,0.0,-0.994789,0.0,0.0,0.0,0.0,0.0,0.0,-0.111148,-5.624082,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.087714,-5.625162,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0,-0.064276,-5.626198,-0.994789,0.0,-0.0,0.0,0.0,0.0,0.0


### Estados repetidos

En este caso se eliminan estados repetidos y estados que se encuentren en estado transitorio mientras el dron despega o se estabiliza antes de introducir la señal de control.

In [ ]:
shape_b4 = dataset.drop(["timestamps"], axis=1).shape
shape_drop= dataset.drop(["timestamps"], axis=1).drop_duplicates().shape
print(f'shape (b4 drop) = {shape_b4}')
print(f'shape = {shape_drop}')
print(f'len (b4 drop) - len = {shape_b4[0]-shape_drop[0]}')

### División del dataset en estados y acciones

In [ ]:
actions = dataset.drop(["timestamps"], axis=1).drop_duplicates()[rpm_list]
print(f'columns = {actions.columns}')
print(f'shape = {actions.shape}')
actions.head()

#### Normalización de acciones

In [ ]:
def normalize_df(df, K=21666.4475, B=14468.4292):
    df_norm = (actions-B)/K
    return df_norm, K, B

In [ ]:
actions, K, B = normalize_df(actions)
actions.describe()

#### Definimos los estados

In [ ]:
states = dataset.drop(["timestamps"], axis=1).drop_duplicates()[states_list]
print(f'columns = {states.columns}')
print(f'shape = {states.shape}')
states.describe()

In [ ]:
del dataset

### Dividimos el dataset

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(states, actions, test_size=0.01)
ins = states.shape[1]
outs = actions.shape[1]
del states, actions

## Keras Model

#### Early Stopping

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

#### Plotting

In [ ]:
class PlotLosses(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.yscale('log')
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

#### Definición del Modelo

In [ ]:
inputs = tf.keras.Input(shape=(ins,))
x = tf.keras.layers.Dense(8*ins, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(4*ins, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(2*ins, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(outs, activation=tf.nn.tanh)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

#### Compilado el Modelo

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0025)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_squared_error'])

#### Entrenamiento del Modelo

In [ ]:
model.fit(X_train, Y_train, validation_split=0.2, epochs=250, batch_size=50000, callbacks=[callback, plot_losses])

#### Evaluación del Modelo

In [ ]:
loss, mean_sq = model.evaluate(X_test, Y_test)
print(f'mean_sq: {mean_sq} -> {mean_sq*K} RPM')
print(f'loss: {loss} -> {loss*K} RPM')

#### Se guarda el Modelo

In [ ]:
I = 'Pandas'
model.save(f'../Models/{dataset_name}_{I}.h5')

In [ ]:
model = tf.keras.models.load_model(f'../Models/{dataset_name}_{I}.h5')
model.summary()

In [ ]:
%%time
x_test = X_test.sample(n=10, random_state=1)
for index, sample in x_test.iterrows():
    print(model.predict([list(sample)])*K+B)

In [ ]:
%%time
x_test = [0]*len(states_list)
x_test[0] = 1
x_test[1] = -9.8
x_test[2] = 3
print(model.predict([list(x_test)])*K+B)